# Illustrates how to program the trigger:

### Connect:

In [ ]:
import phywhisperer.usb as pw
phy = pw.Usb()
phy.con(program_fpga=True)
phy.set_power_source("host")

### Check bitfile build time:
During development it's handy to confirm that the newest bitfile is being used!

In [ ]:
print(phy.get_fpga_buildtime())

### Power-off target to allow PhyWhisperer to autodetect its speed:
Ensure the target device is connected to the PhyWhisperer.
We'll turn off the target's power so that the PhyWhisperer can be programmed before turning the target back on. We do this because we'll be capturing what the target does when it's first turned on.

In [ ]:
import time
phy.set_power_source("off")
time.sleep(0.5)
phy.reset_fpga()

### (Optionally) Manually set USB speed:
By default, PhyWhisperer will auto-detect the target's USB speed when the target is connected or powered-up, so this should not be necessary.

In [ ]:
phy.set_usb_mode('auto')

### Program trigger parameters:
Here we set the capture delay equal to the trigger delay, but they can be set independently:

In [ ]:
phy.set_trigger(delay=0, width=3)
phy.set_capture_delay(delay=0)

### Arm the PhyWhisperer:
You should see the blue ARM LED turn on to reflect the armed status.

In [ ]:
phy.arm()

### Program the pattern match:

In [ ]:
phy.set_pattern(pattern=[0xa5], mask=[0xff])

### Tell PhyWhisperer how many events to capture:
Maximum is 8188.

In [ ]:
phy.set_capture_size(500)

### Power up the target:
Now that PhyWhisperer is programmed, power up the target. PW should auto-detect the correct speed, then the trigger should occur immediately.
The trigger can be observed on the IO4 pin of the ChipWhisperer connector and on the "Trig Out" MCX connector.

In [ ]:
phy.set_power_source("host")
#Let device enumerate
time.sleep(1.0)

### Ensure correct USB speed was detected:
If the assertion fails, try setting the USB speed manually with set_usb_mode().

In [ ]:
assert (phy.get_usb_mode() == 'FS')
#assert (phy.get_usb_mode() == 'LS')
#assert (phy.get_usb_mode() == 'HS')

### Read what was captured:

In [ ]:
raw = phy.read_only_from_fifo(entries=500)

### Interpret the captured data:
The pattern match byte which triggered the captured isn't recorded; let's add it back it so that the USB data can be properly interpreted.
Then, split the raw data into packets, and print them:

In [ ]:
phy.addpattern = True

packets = phy.split_packets(raw)

printPackets = pw.USBSimplePrintSink(highspeed=phy.get_usb_mode() == 'HS')

for packet in packets:
    printPackets.handle_usb_packet(ts=packet['timestamp'], buf=bytearray(packet['contents']), flags=0)

### Iterate and generate a trigger pulse of increasing width:

In [ ]:
import time
start_time = time.time()
for i in range (16):
    print("Iteration %d: " % i, end='')
    # toggle target power on and off:
    phy.set_power_source("off")
    time.sleep(0.1)
    # set trigger parameters:
    phy.set_trigger(delay=0, width=2**i)
    # arm:
    phy.arm()
    phy.set_power_source("host")    
    phy.wait_disarmed()
    print("done")
    time.sleep(0.1)
print("Elapsed time: %d seconds" % (time.time()-start_time))   

In [ ]:
phy.close()